In [1]:
import androguard

from androguard.core import apk, dex
from androguard.core.analysis import analysis
from androguard.misc import logger
import sys
from pathlib import Path
import pickle
import re

from contextlib import suppress


logger.remove()

logger.add(sys.stderr, level="WARNING")



2

In [2]:
print(1 if True else 2)

1


In [3]:
def get_mapping_dex_idx_to_signature(apk_name="org.thoughtcrime.securesms.apk"):
    mapping = {}
    a = apk.APK(apk_name)

    dex_names = a.get_dex_names()

    # fucking androguard looses the dex names at some point and hoping that order is preserved is not my style so we have to manually iterate over the analysis. aaargh
    for dex_name in dex_names:
        dex_bytes = a.get_file(dex_name)
    
        d = dex.DEX(dex_bytes)
        dx = analysis.Analysis(d)

        dex_num = 1 if dex_name == "classes.dex" else int(dex_name.removeprefix("classes").removesuffix(".dex"))

        mapping[dex_num] = {}
    
        for method_analysis in dx.get_methods():
            method = method_analysis.get_method()
            method_idx = method.get_method_idx()
            class_name = method.get_class_name()
            #print(f"{dex_name}, DEX #{i}: {class_name}->{method.get_name()}{method.get_descriptor()}")
            #print(f"{method.full_name}")
            mapping[dex_num][method_idx] = method.full_name

    return mapping

#mapping = get_mapping_dex_idx_to_signature()


In [4]:

def read_from_cache_or_generate_mapping():
    cachepath = Path("mapping_dex_method_idx_to_signature.pickle")
    if not cachepath.exists():
        print(f"no cache found, loading mapping from apk")
        mapping = get_mapping_dex_idx_to_signature()
        with open(cachepath, 'wb') as handle:
            pickle.dump(mapping, handle, protocol=pickle.HIGHEST_PROTOCOL)
    else:
        print(f"loading mapping from cache!")
        with open(cachepath, 'rb') as handle:
            mapping = pickle.load(handle)

    all_methods = set()
    total_num = 0
    for k,v in mapping.items():
        print(f"{k}: {len(v)}")
        total_num += len(v)
        all_methods.update(v.values())
    print(f"total:     {total_num}")
    print(f"total uniq:{len(all_methods)}")
    
    return mapping

mapping = read_from_cache_or_generate_mapping()


loading mapping from cache!
1: 58919
2: 5132
3: 58421
4: 57077
5: 56438
6: 55476
7: 14871
total:     306334
total uniq:306334


In [5]:
def parse_number_line(line, prefix, split, suffix):
    numbers = []
    for number in line.removeprefix(prefix).removesuffix(suffix).split(split):
        numbers.append(int(number))
    return set(numbers)

def profile_txt_to_json(proftxtpath: Path):
    profile_json = {}
    with open(proftxtpath, "r") as f:
        current_dex = -1
        for raw_line in f:
            line = raw_line.strip()
            # skip headers and such
            if (
                line.startswith("===")
                or line.startswith("ProfileInfo")
                or line == ""
            ):
                continue
            # demarks new dexfile entry
            elif "[index=" in line:
                parts = line.split(" ")
                raw_dex = parts[0]
                # the first dex file can be either implied or explicit
                if raw_dex == "base.apk" or raw_dex == "classes.dex":
                    current_dex = 1
                else:
                    # all others are always base.apk!classesN.dex
                    current_dex = int(
                        raw_dex.removeprefix("base.apk!")
                        .removeprefix("classes")
                        .removesuffix(".dex")
                    )
                profile_json[current_dex] = {}
            # parse the different method classifications
            elif line.startswith("hot methods:") and line != "hot methods:":
                # hot methods have some more data that we ignore
                line_without_args = re.sub(r"\[[^\]]+\]", "[]", line)
                profile_json[current_dex]["hot"] = parse_number_line(
                    line_without_args, "hot methods: ", "[], ", "[],"
                )
            elif line.startswith("startup methods:") and line != "startup methods:":
                profile_json[current_dex]["startup"] = parse_number_line(
                    line, "startup methods: ", ", ", ","
                )
            elif (
                line.startswith("post startup methods:")
                and line != "post startup methods:"
            ):
                profile_json[current_dex]["post"] = parse_number_line(
                    line, "post startup methods: ", ", ", ","
                )
            # we parse it but don't use it
            elif line.startswith("classes:") and line != "classes:":
                profile_json[current_dex]["classes"] = parse_number_line(
                    line, "classes: ", ",", ","
                )
    return profile_json

In [6]:
cloud_path = Path("org.thoughtcrime.securesms.dm_files/primary.profdump")
base_path  = Path("org.thoughtcrime.securesms.apk_files/assets/dexopt/baseline.profdump")

cloud = profile_txt_to_json(cloud_path)
base  = profile_txt_to_json(base_path)

In [7]:
# dump to files verbose
print("cloud:")
for dexid, methods_by_type in cloud.items():
    #print(f"{dexid}: {methods_by_type.keys()}")
    print(f"  {dexid}:")
    print(f"    hot:     {len(methods_by_type['hot'])}")
    print(f"    startup: {len(methods_by_type['startup'])}")
    print(f"    post:    {len(methods_by_type['post']) if 'post' in methods_by_type.keys() else 0}")
print("baseline:")
for dexid, methods_by_type in base.items():
    #print(f"{dexid}: {methods_by_type.keys()}")
    print(f"  {dexid}:")
    print(f"    hot:     {len(methods_by_type['hot'])}")
    print(f"    startup: {len(methods_by_type['startup'])}")
    print(f"    post:    {len(methods_by_type['post']) if 'post' in methods_by_type.keys() else 0}")

cloud:
  1:
    hot:     24732
    startup: 25021
    post:    0
  2:
    hot:     115
    startup: 113
    post:    0
  3:
    hot:     7196
    startup: 7547
    post:    0
  4:
    hot:     10012
    startup: 10642
    post:    0
  5:
    hot:     4089
    startup: 5703
    post:    0
  6:
    hot:     3908
    startup: 4762
    post:    0
  7:
    hot:     431
    startup: 578
    post:    0
baseline:
  1:
    hot:     24716
    startup: 24718
    post:    25395
  2:
    hot:     112
    startup: 110
    post:    115
  3:
    hot:     7843
    startup: 7964
    post:    8331
  4:
    hot:     11083
    startup: 11213
    post:    11944
  5:
    hot:     5707
    startup: 5721
    post:    8215
  6:
    hot:     5458
    startup: 6028
    post:    6097
  7:
    hot:     683
    startup: 814
    post:    705


In [8]:
count = 0
resultingmethods = [] # (signature, cloud_hot, cloud_startup, cloud_post, base_hot, base_startup, base_post)

set_dexids_with_profilemethods = set(list(cloud.keys()) + list(base.keys()))
for dex_id,methods in mapping.items():
    if not dex_id in set_dexids_with_profilemethods:
        continue

    for method_idx, method_sig in methods.items():
        cloud_hot = cloud_startup = cloud_post = base_hot = base_startup = base_post = False
        with suppress(KeyError):
            cloud_hot = method_idx in cloud[dex_id]['hot']
        with suppress(KeyError):
            cloud_startup = method_idx in cloud[dex_id]['startup']
        with suppress(KeyError):
            cloud_post = method_idx in cloud[dex_id]['post']
        with suppress(KeyError):
            base_hot = method_idx in base[dex_id]['hot']
        with suppress(KeyError):
            base_startup = method_idx in base[dex_id]['startup']
        with suppress(KeyError):
            base_post = method_idx in base[dex_id]['post']
        if any([cloud_hot, cloud_startup, cloud_post, base_hot, base_startup, base_post]):
            resultingmethods.append((method_sig, cloud_hot, cloud_startup, cloud_post, base_hot, base_startup, base_post))
        
            #count += 1
            #if count > 100:
            #    break

print(len(resultingmethods))

55755


In [9]:
with open("resultingmethods.csv", "w") as f:
    for method_sig, cloud_hot, cloud_startup, cloud_post, base_hot, base_startup, base_post in resultingmethods:
        f.write(f"{method_sig}|{cloud_hot}|{cloud_startup}|{cloud_post}|{base_hot}|{base_startup}|{base_post}\n")

In [ ]:
def get_activities(apk_name="org.thoughtcrime.securesms.apk"):
    a = apk.APK(apk_name)
    return a

In [3]:
a = apk.APK("org.thoughtcrime.securesms.apk")


In [5]:
len(a.get_activities())

90

In [6]:
len(a.get_activity_aliases())

12

In [7]:
a.get_activities()

['org.thoughtcrime.securesms.contactshare.ContactShareEditActivity',
 'org.thoughtcrime.securesms.contacts.TurnOffContactJoinedNotificationsActivity',
 'org.thoughtcrime.securesms.maps.PlacePickerActivity',
 'org.thoughtcrime.securesms.ShortcutLauncherActivity',
 'org.thoughtcrime.securesms.MainActivity',
 'org.thoughtcrime.securesms.SmsSendtoActivity',
 'org.thoughtcrime.securesms.groups.ui.chooseadmin.ChooseNewAdminActivity',
 'com.android.billingclient.api.ProxyBillingActivityV2',
 'com.android.billingclient.api.ProxyBillingActivity',
 'org.thoughtcrime.securesms.pin.PinRestoreActivity',
 'org.thoughtcrime.securesms.groups.ui.creategroup.CreateGroupActivity',
 'org.thoughtcrime.securesms.groups.ui.addtogroup.AddToGroupsActivity',
 'org.thoughtcrime.securesms.groups.ui.addmembers.AddMembersActivity',
 'org.thoughtcrime.securesms.megaphone.ClientDeprecatedActivity',
 'org.thoughtcrime.securesms.ratelimit.RecaptchaProofActivity',
 'org.thoughtcrime.securesms.wallpaper.crop.WallpaperIma

In [8]:
a.get_activity_aliases()

[{'name': 'org.thoughtcrime.securesms.RoutingActivityAltWeather',
  'targetActivity': 'org.thoughtcrime.securesms.MainActivity'},
 {'name': 'org.thoughtcrime.securesms.RoutingActivityAltNews',
  'targetActivity': 'org.thoughtcrime.securesms.MainActivity'},
 {'name': 'org.thoughtcrime.securesms.RoutingActivityAltNotes',
  'targetActivity': 'org.thoughtcrime.securesms.MainActivity'},
 {'name': 'org.thoughtcrime.securesms.RoutingActivityAltBubbles',
  'targetActivity': 'org.thoughtcrime.securesms.MainActivity'},
 {'name': 'org.thoughtcrime.securesms.RoutingActivityAltYellow',
  'targetActivity': 'org.thoughtcrime.securesms.MainActivity'},
 {'name': 'org.thoughtcrime.securesms.RoutingActivityAltWaves',
  'targetActivity': 'org.thoughtcrime.securesms.MainActivity'},
 {'name': 'org.thoughtcrime.securesms.RoutingActivity',
  'targetActivity': 'org.thoughtcrime.securesms.MainActivity'},
 {'name': 'org.thoughtcrime.securesms.RoutingActivityAltWhite',
  'targetActivity': 'org.thoughtcrime.secure